In [74]:
import keyring
import pandas as pd
import datetime
import time
import pymssql
from selenium import webdriver
from urllib.request import urlopen
from pymongo import MongoClient
from bs4 import BeautifulSoup

In [2]:
# Get dates for query
today = datetime.datetime.today()
one_week_ago = today - datetime.timedelta(days=7)

In [3]:
# Create connection string
user = "dbadmin"
password = keyring.get_password('docdb_prod_dbadmin', 'dbadmin')
host = "production-documentdb.cluster-cb6kajicuplh.us-east-1.docdb.amazonaws.com"
port = "27017"
options = "tls=true&tlsAllowInvalidCertificates=true&replicaSet=rs0&readPreference=secondaryPreferred&retryWrites=false"
cxn_string_template= "mongodb://{}:{}@{}:{}/?{}"
cxn_string_prod = cxn_string_template.format(user, password, host, port, options)


# Get past week's new products

In [4]:
# Get past week's new products
client = MongoClient(cxn_string_prod)
db = client['product-prod']
results_ = list(db.PdwProductCore.find({'createTimestamp': {'$gte': one_week_ago}}, 
                                       ('productGeneral.cusip', 'productGeneral.isin')))
cusips = [i['productGeneral']['cusip'] for i in results_ if 'cusip' in i['productGeneral']]
isins = [i['productGeneral']['isin'] for i in results_ if 'isin' in i['productGeneral']]

# Get BMO products added within first 5 pages

In [5]:
# # Setup for bmo
# bmo_act_dict = {}
# chrome_path = r"/opt/homebrew/bin/chromedriver"
# op = webdriver.ChromeOptions()
# op.add_argument('--headless')
# urls = [
#     'https://www.bmonotes.com/Type/PPNs#active', 
#     'https://www.bmonotes.com/Type/Fixed-Income-Notes#active', 
#     'https://www.bmonotes.com/Type/NPPNs#active'
# ]
# all_bmo_active_products = pd.DataFrame()
# for url in urls:
#     print(url)
#     # Get first page
#     driver = webdriver.Chrome(chrome_path, options=op)
#     page = driver.get(url)
#     time.sleep(2)
#     bmo_act_dict[0] = pd.read_html(driver.page_source)[1]
#     # Get remaining pages
#     flag = True
#     num = 1
#     while flag:
#         if 'disabled' in driver.find_element_by_id("DataTables_Table_1_next").get_attribute('class'):
#             flag = 0
#         else:
#             driver.find_element_by_id("DataTables_Table_1_next").click()
#             time.sleep(2)
#             bmo_act_dict[num] = pd.read_html(driver.page_source)[1]
#             num += 1
#     driver.close()
#     # Combine the dataframes
#     bmo_active_products = pd.concat([bmo_act_dict[k] for k in bmo_act_dict.keys()], ignore_index=True)
#     all_bmo_active_products = pd.concat([all_bmo_active_products, bmo_active_products], ignore_index=True)
# # Remove junk
# all_bmo_active_products = all_bmo_active_products[(all_bmo_active_products['JHN Code / Cusip'].isna()==False) & (all_bmo_active_products['JHN Code / Cusip']!='Loading...')]
# # Create pdw cusip for comparison
# listy = []
# for i in all_bmo_active_products['JHN Code / Cusip']:
#     if 'JHN' in str(i):
#         if len(str(i)) == 7:
#             listy.append('CA' + str(i))
#         elif len(str(i)) == 8:
#             listy.append('C' + str(i))
#         elif len(str(i)) == 6:
#             listy.append('CAA' + str(i))
#         else:
#             listy.append('Error')
#     else:
#         if len(str(i)) == 9:
#             listy.append(str(i))
#         else:
#             listy.append('Error')
# all_bmo_active_products['pdwCusip'] = listy
# # Filter out any products already present in pdw
# new_bmo_active_products = all_bmo_active_products[~all_bmo_active_products['JHN Code / Cusip'].isin(cusips) | 
#                                               all_bmo_active_products['JHN Code / Cusip'].isin(isins)]

# Testing NBCSS

In [6]:
# # Setup for nbc_ss
# nbc_ss_act_dict = {}
# chrome_path = r"/opt/homebrew/bin/chromedriver"
# op = webdriver.ChromeOptions()
# op.add_argument('--headless')
# driver = webdriver.Chrome(chrome_path, options=op)
# url = 'https://www.nbcstructuredsolutions.ca/listeProduits.aspx?mode=previous'
# # Get first page
# page = driver.get(url)
# time.sleep(2)
# nbc_ss_act_dict[0] = pd.read_html(driver.page_source)[0]
# nbc_ss_act_dict[0]['urls'] = [driver.find_element_by_xpath('//*[@id="ctl00_cphMain_lvProducts_ctrl{}_lnkProduit"]'.format(i)).get_attribute('href') for i in range(len(nbc_ss_act_dict[0]))]
# # Get remaining 4 pages
# for num in range(2, 6):
#     driver.find_element_by_xpath('//*[@id="ctl00_cphMain_dpProductsHaut"]/a[{}]'.format(str(num))).click()
#     time.sleep(2)
#     nbc_ss_act_dict[num] = pd.read_html(driver.page_source)[0]
#     nbc_ss_act_dict[num]['urls'] = [driver.find_element_by_xpath('//*[@id="ctl00_cphMain_lvProducts_ctrl{}_lnkProduit"]'.format(i)).get_attribute('href') for i in range(len(nbc_ss_act_dict[num]))]
# driver.close()
# # Combine the dataframes
# nbc_ss_active_products = pd.concat([nbc_ss_act_dict[k] for k in nbc_ss_act_dict.keys()], ignore_index=True)


In [7]:
# nbc_ss_active_products

# Testing rbcnotes

In [8]:
# rbc_act_dict = {}
# chrome_path = r"/opt/homebrew/bin/chromedriver"
# op = webdriver.ChromeOptions()
# op.add_argument('--headless')
# driver = webdriver.Chrome(chrome_path, options=op)
# url = 'https://www.rbcnotes.com/Products'
# # Get first page
# page = driver.get(url)
# time.sleep(2)
# rbc_act_dict[0] = pd.read_html(driver.page_source)

actual_rbc_data = rbc_act_dict[0][1][(rbc_act_dict[0][1][7].isna()==False) & (rbc_act_dict[0][1][7].str.contains('Day')==False)]
actual_rbc_data.columns = rbc_act_dict[0][0].columns
actual_rbc_data[['Product Name', 'FundSERV Code', 'ADP Code', 'CUSIP', 'End of Day Price', 'Current ETC', 'ETC End Date', 'Issue Date', 'Maturity Date', 'Currency']]

In [9]:
# # Setup for rbc
# rbc_act_dict = {}
# chrome_path = r"/opt/homebrew/bin/chromedriver"
# op = webdriver.ChromeOptions()
# op.add_argument('--headless')
# driver = webdriver.Chrome(chrome_path, options=op)
# url = 'https://www.rbcnotes.com/Products'
# # Get first page
# page = driver.get(url)
# time.sleep(2)
# temp_table = pd.read_html(driver.page_source)
# temp_data = temp_table[1][(temp_table[1][7].isna()==False) & (temp_table[1][7].str.contains('Day')==False)]
# temp_data.columns = temp_table[0].columns
# rbc_act_dict[0] = temp_data
# rbc_act_dict[0]['urls'] = [driver.find_element_by_xpath('//*[@id="productGrid"]/div[2]/div[3]/table/tbody/tr[{}]/td[3]/a'.format(i)).get_attribute('href') 
#                                 for i in range(1, len(rbc_act_dict[0])*2, 2)]
# # Get remaining 4 pages
# for num in range(2, 6):
#     # Scroll to bottom of page to avoid cookie consent form
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     driver.find_element_by_xpath('//*[@id="productGrid"]/div[2]/div[4]/a[3]').click()
#     time.sleep(2)
#     temp_table = pd.read_html(driver.page_source)
#     temp_data = temp_table[1][(temp_table[1][7].isna()==False) & (temp_table[1][7].str.contains('Day')==False)]
#     temp_data.columns = temp_table[0].columns
#     rbc_act_dict[num] = temp_data
#     rbc_act_dict[num]['urls'] = [driver.find_element_by_xpath('//*[@id="productGrid"]/div[2]/div[3]/table/tbody/tr[{}]/td[3]/a'.format(i)).get_attribute('href') 
#                                 for i in range(1, len(rbc_act_dict[0])*2, 2)]
# driver.close()
# # Combine the dataframes
# rbc_active_products = pd.concat([rbc_act_dict[k] for k in rbc_act_dict.keys()], ignore_index=True)
# rbc_active_products = rbc_active_products[['Product Name', 'FundSERV Code', 'ADP Code',
#        'CUSIP', 'End of Day Price', 'Current ETC', 'ETC End Date',
#        'Issue Date', 'Maturity Date', 'Currency', 'urls']]


# Testing Desjardins

In [106]:
# Setup for Desjardins
desj_act_dict = {}
chrome_path = r"/opt/homebrew/bin/chromedriver"
op = webdriver.ChromeOptions()
# op.add_argument('--headless')
op.add_argument('--window-size=1920,1080')
op.add_experimental_option( "prefs",{'profile.managed_default_content_settings.javascript': 1})
driver = webdriver.Chrome(chrome_path, options=op)
url = 'https://www.fondsdesjardins.com/structurednotes/products/index.jsp'
# Get non-paginated PPN page
page = driver.get(url)
time.sleep(2)
# driver.execute_script("arguments[0].click();", driver.find_element_by_class_name('en translate-link first hide'))
driver.execute_script("arguments[0].click();", driver.find_element_by_xpath('//*[@id="splash-page"]/div/div/div[2]/div[2]/div[3]/button'))
time.sleep(20)
# print(driver.page_source)
html_table = BeautifulSoup(driver.page_source).find('table')
ppn_table = pd.read_html(driver.page_source)[0]
ppn_table['links'] = [link.get('href') for link in html_table.find_all('a') if '/pdf/' not in link.get('href')]
driver.execute_script("arguments[0].click();", driver.find_element_by_link_text('Non-Principal Protected'))
time.sleep(2)
nppn_table = pd.read_html(driver.page_source)[1]
html_table = BeautifulSoup(driver.page_source).find_all('table')[1]
nppn_table['links'] = [link.get('href') for link in html_table.find_all('a') if '/pdf/' not in link.get('href')]

In [107]:
nppn_table

,Name,Code,Type,Reference Portfolio,Term,Issue Date,Days Remaining,Documents (PDF),links
0,Desjardins Fixed Coupon Notes (Maturity-Monito...,DSN00393,Income,Diversified American Equities,"5 years, 11 months, 30 days",09/16/2022,11 days,Client Summary Pricing Supplement Prospectus...,/structurednotes/product/fixed-coupon-notes-(m...
1,Desjardins Fixed Coupon Notes (Maturity-Monito...,DSN00394,Income,Diversified American Equities,"5 years, 11 months, 30 days",09/16/2022,11 days,Client Summary Pricing Supplement Prospectus...,/structurednotes/product/fixed-coupon-notes-(m...
2,Desjardins Fixed Coupon Notes (Maturity-Monito...,DSN00392,Income,Diversified American Equities,"5 years, 11 months, 30 days",08/26/2022,0 days,Client Summary Pricing Supplement Prospectus...,/structurednotes/product/fixed-coupon-notes-(m...
3,Desjardins Fixed Coupon Notes (Maturity-Monito...,DSN00391,Income,Diversified American Equities,"5 years, 11 months, 30 days",08/26/2022,0 days,Client Summary Pricing Supplement Prospectus...,/structurednotes/product/fixed-coupon-notes-(m...


# Testing Nova Scotia

In [ ]:
# # Setup for Nova Scotia
# nosco_act_dict = {}
# chrome_path = r"/opt/homebrew/bin/chromedriver"
# op = webdriver.ChromeOptions()
# op.add_argument('--headless')
# driver = webdriver.Chrome(chrome_path, options=op)
# url = 'https://www.investorsolutions.gbm.scotiabank.com/ppn-public/home.do'